In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [14]:
import PyPDF2
import re
import pickle

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        str: Extracted text from the PDF.
    """
    try:
        text = ""
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            num_pages = len(pdf_reader.pages)

            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()

        return text

    except Exception as e:
        return f"Error: {str(e)}"

def process_text(text):
    """
    Trova tutti gli elementi che corrispondono al pattern tipico di un indice:
    testo seguito da una serie di punti e un numero di pagina.

    Args:
        testo (str): Il testo in cui cercare il pattern

    Returns:
        list: Lista di tuple (titolo, numero_pagina)
    """
    # Pattern regex: testo seguito da almeno 5 punti e poi un numero, escludendo titoli che iniziano con un numero e un punto
    pattern = r'(?<!\d\.\s)(.+?)\s*\.{5,}\s*(\d+)'

    # Trova tutte le corrispondenze
    matches = re.findall(pattern, text, re.MULTILINE)

    # Pulisci i risultati (rimuovi spazi extra)
    results = [(title.strip(), int(page)) for title, page in matches if re.match(r'\b\w+(\s+\w+)+\b', title)]

    return results


def cut_text_before_phrase(text, phrase):
    """
    Cuts all text before the specified phrase.

    Args:
        text (str): The original text.
        phrase (str): The phrase to search for.

    Returns:
        str: The text starting from the specified phrase.
    """
    # Find the index of the phrase in the text
    index = text.find(phrase)

    # If the phrase is found, return the text starting from that phrase
    if index != -1:
        return text[index:]
    else:
        # If the phrase is not found, return the original text
        return text


def dividi_testo_per_titoli(testo, lista_titoli):
    """
    Divide il testo in chunk in base ai titoli forniti in una lista di tuple.
    Considera un titolo valido solo se è preceduto da un carattere newline.

    Args:
        testo (str): Il testo completo da dividere
        lista_titoli (list): Lista di tuple (titolo, livello) dove titolo è il pattern da cercare

    Returns:
        list: Lista di dizionari, ciascuno contenente il titolo, il livello e il contenuto del chunk
    """
    chunks = []

    # Estrai solo i titoli dalle tuple e crea un dizionario di mappatura titolo -> livello
    titoli_livelli = {titolo: livello for titolo, livello in lista_titoli}

    # Ordina i titoli dal più lungo al più corto per evitare matching parziali
    titoli_ordinati = sorted(titoli_livelli.keys(), key=len, reverse=True)

    # Assicurati che il testo inizi con un newline per catturare titoli all'inizio
    testo_con_newline = '\n' + testo if not testo.startswith('\n') else testo

    # Trova tutte le posizioni dei titoli nel testo
    matches = []
    for titolo in titoli_ordinati:
        # Cerca il titolo ma solo se viene dopo un newline
        pattern = r'\n' + re.escape(titolo)
        for match in re.finditer(pattern, testo_con_newline):
            # Memorizza il titolo e la posizione
            matches.append((match.start(), match.end(), titolo))

    # Ordina i match per posizione
    matches.sort(key=lambda x: x[0])

    # Se non ci sono match, restituisci una lista vuota
    if not matches:
        return chunks

    # Processa ogni match per dividere il testo
    for i, (start, end, titolo_trovato) in enumerate(matches):
        # L'inizio effettivo del contenuto è dopo il newline + titolo
        inizio = start + 1  # +1 per saltare il newline iniziale

        # Calcola la fine della sezione (inizio della sezione successiva o fine del testo)
        fine = matches[i+1][0] if i < len(matches) - 1 else len(testo_con_newline)

        # Estrai il testo per questa sezione
        contenuto = testo_con_newline[inizio:fine].strip()

        # Crea il chunk e aggiungilo alla lista
        chunk = {
            'title': titolo_trovato,
            'content': contenuto
        }
        chunks.append(chunk)

    return chunks



# Example usage
if __name__ == "__main__":
    pdf_path = "/content/drive/MyDrive/ELAN_manual.pdf"  # Replace with your PDF file path
    extracted_text = extract_text_from_pdf(pdf_path)
    titles = process_text(extracted_text)
    cleaned_text = cut_text_before_phrase(extracted_text, "xxiChapter 1. ELAN documents")
    chunks = dividi_testo_per_titoli(cleaned_text, titles)
    chunks_cleaned = [el for el in chunks if el['title'] != el['content']]
    with open('/content/drive/MyDrive/ELAN_chunks.pkl', 'wb') as file:
      pickle.dump(chunks_cleaned, file)



